# Model Evaluation Script

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from time import localtime
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras import regularizers, optimizers
from keras.callbacks import EarlyStopping, TensorBoard, ProgbarLogger
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV

from sklearn import ensemble
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression


from keras import backend as K
from sklearn.metrics import log_loss




# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../NCAA"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


Data
Data_Organization.ipynb
GiantKillerInfo.txt
MarchMadnessAdvStats.csv
MarchMadnessFeatureDifferences.csv
MarchMadnessFeatures.csv
MarchMadnessTest.csv
ModelEvaluation.ipynb
OldNbs
SubmissionData.py
Submissions
logs
tENSORFLOW-TB.ipynb
tENSORFLOW.ipynb
tENSORFLOW.py



Lock Random Seed

In [2]:
# seed = 13
# np.random.seed(seed)

## Load pre-organized data

In [3]:
data_dir = '../NCAA/'
df_features = pd.read_csv(data_dir + 'MarchMadnessFeatures.csv')

Format and scale the data

In [4]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components = 2)
# X = pca.fit(X)

In [5]:
X = df_features.iloc[:,1:]
xDim = np.shape(X)[1]
X_train = X.values.reshape(-1,xDim)
y_train = df_features.Result.values

scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)


print('Feature vector dimension is: %.2f' % xDim)
# print(X[:5, :])

Feature vector dimension is: 34.00


# Load Training Data


In [6]:
df_test = pd.read_csv(data_dir+'MarchMadnessTest.csv')

X_test = df_test.iloc[:,1:]
xDimTest = np.shape(X_test)[1]
X_test = X_test.values.reshape(-1,xDimTest)
y_test = df_test.Result.values

# X_test = scaler.transform(X_test)

Create Kfold splits

In [9]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)

## MLP

Various properties

In [12]:
dropRate = 0.3
numBatch = 50
numEpoch = 120
learningRate = 1e-4

Base Single Layer Model

In [13]:
# MLP model
MLP = Sequential()
MLP.name = 'MLP'
MLP.add(Dense(50, input_dim=xDim, kernel_initializer='random_normal',activation = 'tanh'))
MLP.add(Dropout(dropRate))
# MLP.add(BatchNormalization())
MLP.add(Dense(500,kernel_initializer='normal',activation = 'tanh'))
MLP.add(Dropout(dropRate))
# # MLP.add(BatchNormalization())
# MLP.add(Dense(100, kernel_initializer='normal',activation = 'tanh'))
# MLP.add(Dropout(dropRate))
# # MLP.add(BatchNormalization())
MLP.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

# Compile model
adam = optimizers.Adam(lr=learningRate, amsgrad=True)
MLP.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [14]:
# TB = TensorBoard(log_dir="logs/{}_{}{}".format(MLP.name, localtime().tm_hour, localtime().tm_min))
# MLP.fit(X, y, validation_split = 0.2, epochs=numEpoch, batch_size=numBatch, verbose=0, callbacks = [TB])
MLP.fit(X_train, y_train, epochs=numEpoch, batch_size=numBatch, verbose=1)


Epoch 1/120
530/530 [==============================] - 1s 1ms/step - loss: 0.6456 - acc: 0.6226
Epoch 2/120
530/530 [==============================] - 0s 74us/step - loss: 0.6518 - acc: 0.6245
Epoch 3/120
530/530 [==============================] - 0s 78us/step - loss: 0.6338 - acc: 0.6396
Epoch 4/120
530/530 [==============================] - 0s 76us/step - loss: 0.6424 - acc: 0.6340
Epoch 5/120
530/530 [==============================] - 0s 81us/step - loss: 0.6346 - acc: 0.6604
Epoch 6/120
530/530 [==============================] - 0s 87us/step - loss: 0.6087 - acc: 0.6604
Epoch 7/120
530/530 [==============================] - 0s 85us/step - loss: 0.6122 - acc: 0.6792
Epoch 8/120
530/530 [==============================] - 0s 74us/step - loss: 0.6106 - acc: 0.6868
Epoch 9/120
530/530 [==============================] - 0s 81us/step - loss: 0.6143 - acc: 0.6528
Epoch 10/120
530/530 [==============================] - 0s 89us/step - loss: 0.6261 - acc: 0.6698
Epoch 11/120
530/530 [========

In [15]:
MLP.evaluate(X_test, y_test)

402/402 [==============================] - 0s 135us/step


[0.53914516274608781, 0.72139303452933012]

## Making predictions with model

### Extract data desired

In [30]:
data_dir = '../NCAA/Data/'
df_sample_sub = pd.read_csv(data_dir + 'SampleSubmissionStage1.csv')
data_file = data_dir + 'MarchMadnessAdvStats.csv'
df_adv = pd.read_csv(data_file)
df_seeds = pd.read_csv(data_dir + 'NCAATourneySeeds.csv')


n_test_games = len(df_sample_sub)

def get_year_t1_t2(ID):
    """Return a tuple with ints `year`, `team1` and `team2`."""
    return (int(x) for x in ID.split('_'))

def seed_to_int(seed):
    '''Get just the digits from the seeding. Return as int'''
    s_int = int(seed[1:3])
    return s_int

In [ ]:
print('Loading data for submission test')

# Make the seeding an integer
df_seeds['seed_int'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(columns=['Seed'], inplace=True) # This is the string label
df_seeds.head()


T1_seed = []
T1_adv = []
T2_adv = []
T2_seed = []
for ii, row in df_sample_sub.iterrows():
    year, t1, t2 = get_year_t1_t2(row.ID)
    t1_seed = df_seeds[(df_seeds.TeamID == t1) & (df_seeds.Season == year)].seed_int.values[0]
    t2_seed = df_seeds[(df_seeds.TeamID == t2) & (df_seeds.Season == year)].seed_int.values[0]
    t1_adv = df_adv[(df_adv.TeamID == t1) & (df_adv.Season == year)].values[0]
    t2_adv = df_adv[(df_adv.TeamID == t2) & (df_adv.Season == year)].values[0]
    T1_seed.append(t1_seed)
    T1_adv.append(t1_adv)
    T2_seed.append(t2_seed)
    T2_adv.append(t2_adv)

T1_adv = [row[2:] for row in T1_adv]
T2_adv = [row[2:] for row in T2_adv]
T1_seed = np.reshape(T1_seed, [n_test_games,-1]).tolist()
T2_seed = np.reshape(T2_seed, [n_test_games, -1]).tolist()
X_pred = np.concatenate((T1_seed, T1_adv, T2_seed, T2_adv), axis=1)

df_subData = pd.DataFrame(np.array(X_pred).reshape(np.shape(X_pred)[0], np.shape(X_pred)[1]))

xDim = np.shape(df_subData)[1]
X_pred = df_subData.values.reshape(-1,xDim)

In [36]:
np.shape(T1_seed)

(9112, 1)

In [33]:
# X_test = scaler.fit_transform(X_test)

#s*************Check if this works!!!
preds = MLP.predict_proba(X_pred)

# df_sample_sub = pd.DataFrame()
# clipped_preds = np.clip(preds, 0.05, 0.95)
df_sample_sub.Pred = preds
df_sample_sub.shape

(9112, 2)

In [26]:
filename = 'MLP'
save_dir = '../NCAA/Submissions/'
c=0
ext = '.csv'
if os.path.exists(save_dir+filename+ext):
    while os.path.exists(filename+ext):
        c+=1
    filename = filename+'_'+str(c)
    df_sample_sub.to_csv(save_dir+filename+ext, index=False)
else:
    df_sample_sub.to_csv(save_dir+filename+ext, index=False)